In [1]:
import numpy as np
import os 
from utils import slice_dataframe
from helper import create_lstm_autoencoder, load_valid_norm_datasets_sorted_by_date, split_dataframe_train_test
import tensorflow as tf

In [2]:
# device checking
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) # tensorflow >=2.4.0
print(tf.sysconfig.get_build_info())


False
[]
OrderedDict([('is_cuda_build', False), ('is_rocm_build', False), ('is_tensorrt_build', False)])


In [3]:
# 데이터를 로드 합니다.
_, norm_target_df, scalers = load_valid_norm_datasets_sorted_by_date('./pu_batt_sample10.csv')

FileNotFoundError: [Errno 2] No such file or directory: './pu_batt_sample10.csv'

In [3]:
# 데이터 일정한 크기와 간격으로 slice 합니다.
steps = 10
strides = 3
n_sample = None
cache_name = 'sliced_dfs.npy'
if not os.path.exists(cache_name):
    sliced_dfs = slice_dataframe(norm_target_df.iloc[:n_sample], interval=steps, stride=strides, output_type='numpy')
else:
    # load cache
    print('Load Cached file')
    sliced_dfs = np.load(cache_name)
    print('shape ', sliced_dfs.shape)
    
    

100%|██████████| 658491/658491 [02:05<00:00, 5242.57it/s]


<function helper.split_dataframe_train_test(df, train_ratio=0.7)>

In [4]:
# 예제 입력 및 출력 차원 및 은닉 유닛 수
input_n_features = norm_target_df.shape[-1]
hidden_units = 64

# 모델 생성
model = create_lstm_autoencoder(steps, input_n_features, hidden_units)

# 모델 요약
model.summary()
model.compile('adam', loss='mse')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 100)]            0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 10, 64),             42240     ['input_1[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 lstm_1 (LSTM)               [(None, 10, 64),             33024     ['lstm[0][0]',                
                              (None, 64),                            'lstm[0][1]',            

In [5]:
sliced_dfs = np.array(sliced_dfs)
model.fit(sliced_dfs, sliced_dfs, epochs=100)

Epoch 1/100
 1597/20578 [=>............................] - ETA: 2:08 - loss: 0.0044

KeyboardInterrupt: 

In [15]:
y_hat = model.predict(sliced_dfs)

11/11 [==============================] - 0s 4ms/step


In [18]:
diff = sliced_dfs - y_hat